# Building a Spam Filter with Naive Bayes

Aim to use the multinomial Naive Bayes algorithm build a spam filter for SMS messages.

To train the algorithm, we'll use a dataset of 5,572 SMS messages that have been manually classified. 

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [2]:
raw_data = pd.read_csv(
    '~/data/smsspamcollection/SMSSpamCollection',
    sep='\t', 
    header=None, 
    names=['Label', 'SMS']
)

In [3]:
raw_data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
raw_data.shape

(5572, 2)

In [5]:
# Here we see that around 13% of the messages are spam. 
raw_data['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## Rename labels (optional - just personal preference)

In [6]:
data = raw_data.apply(lambda x : x.replace('ham','non-spam'))
data.head()

,Label,SMS
0,non-spam,"Go until jurong point, crazy.. Available only ..."
1,non-spam,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,non-spam,U dun say so early hor... U c already then say...
4,non-spam,"Nah I don't think he goes to usf, he lives aro..."


## Randomise and split in to train and test data

In [7]:
# Randomize the dataset
data_randomised = data.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomised) * 0.8)

# Training/Test split
data_train = data_randomised[:training_test_index].reset_index(drop=True)
data_test = data_randomised[training_test_index:].reset_index(drop=True)

print(data_train.shape)
print(data_test.shape)

(4458, 2)
(1114, 2)


In [8]:
data_train['Label'].value_counts(normalize=True)

non-spam    0.86541
spam        0.13459
Name: Label, dtype: float64

In [9]:
data_test['Label'].value_counts(normalize=True)

non-spam    0.868043
spam        0.131957
Name: Label, dtype: float64

## Clean Training Data

In [10]:
data_train.head()

,Label,SMS
0,non-spam,"Yep, by the pretty sculpture"
1,non-spam,"Yes, princess. Are you going to make me moan?"
2,non-spam,Welp apparently he retired
3,non-spam,Havent.
4,non-spam,I forgot 2 ask ü all smth.. There's a card on ...


In [11]:
# Remove punctuation then lowercase all characters
data_train['SMS'] = data_train['SMS'].str.replace('\W', ' ')
data_train['SMS'] = data_train['SMS'].str.lower()
data_train.head()

,Label,SMS
0,non-spam,yep by the pretty sculpture
1,non-spam,yes princess are you going to make me moan
2,non-spam,welp apparently he retired
3,non-spam,havent
4,non-spam,i forgot 2 ask ü all smth there s a card on ...


## Create a vocabulary for all the messages in the traing data

Vocabulary, in this context, means a list with all the unique words in our training set.

In [12]:
data_train['SMS'] = data_train['SMS'].str.split()

vocabulary = []

for item in data_train['SMS']:
    for word in item:
        vocabulary.append(word)  

vocabulary = list(set(vocabulary))

## Create word count dataframe

In [13]:
word_counts_per_sms = {unique_word: [0] * len(data_train['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(data_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [14]:
word_counts = pd.DataFrame(word_counts_per_sms)

In [15]:
train_data_final = pd.concat([data_train, word_counts], axis=1)

In [16]:
train_data_final.head()

,Label,SMS,caroline,always,carry,fly,greeting,education,1win150ppmx3,apps,...,filling,mr,camera,smoothly,funs,town,08715203652,fever,spending,09053750005
0,non-spam,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,non-spam,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,non-spam,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,non-spam,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,non-spam,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants

The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

We'll also use Laplace smoothing and set $\alpha = 1$.

In [17]:
# Isolating spam and non-spam messages first
spam_messages = train_data_final[train_data_final['Label'] == 'spam']
non_spam_messages = train_data_final[train_data_final['Label'] == 'non-spam']

# P(spam) and P(non-spam)
p_spam = len(spam_messages) / len(train_data_final)
p_non_spam = len(non_spam_messages) / len(train_data_final)

# number of words in spam messages
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# number of words in non-spam messages
n_words_per_non_spam_message = non_spam_messages['SMS'].apply(len)
n_non_spam = n_words_per_non_spam_message.sum()

# Number of words in Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calculating Parameters

Calculate the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [18]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_non_spam = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_non_spam = non_spam_messages[word].sum()
    p_word_given_non_spam = (n_word_given_non_spam + alpha) / (n_non_spam + alpha*n_vocabulary)
    parameters_non_spam[word] = p_word_given_non_spam

## Classifying A New Message

The spam filter can be understood as a function that:

- Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    -  If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [19]:
def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_non_spam_given_message = p_non_spam

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_non_spam:
            p_non_spam_given_message *= parameters_non_spam[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Non Spam|message):', p_non_spam_given_message)
    
    if p_non_spam_given_message > p_spam_given_message:
        print('Label: Non Spam')
    elif p_non_spam_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [20]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Non Spam|message): 3.687530435009238e-21
Label: Non Spam


In [21]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Non Spam|message): 1.9368049028589875e-27
Label: Spam


## Measuring the Spam Filter's Accuracy

Write a function that returns classification labels instead of printing them.

In [22]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_non_spam_given_message = p_non_spam

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_non_spam:
            p_non_spam_given_message *= parameters_non_spam[word]
    
    if p_non_spam_given_message > p_spam_given_message:
        return 'non-spam'
    elif p_spam_given_message > p_non_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Use function to create a new column in our test set.

In [23]:
data_test['predicted'] = data_test['SMS'].apply(classify_test_set)
data_test.head()

,Label,SMS,predicted
0,non-spam,Later i guess. I needa do mcat study too.,non-spam
1,non-spam,But i haf enuff space got like 4 mb...,non-spam
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,non-spam,All sounds good. Fingers . Makes it difficult ...,non-spam
4,non-spam,"All done, all handed in. Don't know if mega sh...",non-spam


Write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [24]:
def validate(df):
    
    correct = 0
    total = df.shape[0]

    for row in df.iterrows():
        row = row[1]
        if row['Label'] == row['predicted']:
            correct += 1

    print('Correct:', correct)
    print('Incorrect:', total - correct)
    print('Accuracy:', correct/total)

In [25]:
validate(data_test)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.